# Dados

In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import cv2
import shutil
from module import data_read, setup
from torch.utils.data import ConcatDataset
from module.LightningModule import ClassificationModule
from module.setup import *
from module.data_read import *
from module.plot import imshow
from module import data_read
from torchvision import models
from torch import nn
import lightning as L
from lightning.pytorch.callbacks import Timer
from sklearn.model_selection import KFold
from lightning.pytorch.loggers import TensorBoardLogger

torch.manual_seed(random_state)
torch.set_float32_matmul_precision('high')

DDR_DIR = Path.cwd() / 'DDR-dataset/DR_grading'
IDRID_DIR = Path.cwd() / 'IDRID-Classificacao'

# Lighting Module

## Binary

In [2]:
def binary_train_and_test(model, model_name):

    anno_train_ddr = pd.read_csv(DDR_DIR/'train.txt', header=None, sep=' ')
    anno_train_ddr[0] = 'train/'+anno_train_ddr[0]
    anno_valid_ddr = pd.read_csv(DDR_DIR/'valid.txt', header=None, sep=' ')
    anno_valid_ddr[0] = 'valid/'+anno_valid_ddr[0]
    anno_test_ddr = pd.read_csv(DDR_DIR/'test.txt', header=None, sep=' ')
    anno_test_ddr[0] = 'test/'+anno_test_ddr[0]
    
    # Todas as anotações do DDR
    anno_ddr = pd.concat([anno_train_ddr, anno_valid_ddr, anno_test_ddr], ignore_index=True)
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=random_state)
    
    for fold, (train_ids, test_ids) in enumerate(kfold.split(anno_ddr)):
        print(f'FOLD {fold}')
        print('--------------------------------')
        
        # Divide os dados com base nos atuais indices do K-fold
        train_dataset = DDRDatasetKFold(DDR_DIR, anno_ddr.iloc[train_ids], data_transforms['train'], convert_to_binary=True, dataset_name='ddr')
        test_dataset = DDRDatasetKFold(DDR_DIR, anno_ddr.iloc[test_ids], data_transforms['test'], convert_to_binary=True, dataset_name='ddr')
    
    
        # Definição dos dataloaders
        trainloader = torch.utils.data.DataLoader(
                          train_dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=8) # MUDAR
        testloader = torch.utils.data.DataLoader(
                          test_dataset,
                          batch_size=batch_size,
                          shuffle=False,
                          num_workers=8)
        idridloader = torch.utils.data.DataLoader(
                          idrid_dataset,
                          batch_size=batch_size,
                          num_workers=8)
    
    
    
        logger = TensorBoardLogger(f'{CHECKPOINTS_DIR}/tb_logs', name=f"FOLD {fold} {model_name}")
        timer = Timer()
        # DEBUG:
        # trainer = L.Trainer(fast_dev_run=8, callbacks=[timer]) # a execução do trainer se limitará a fast_dev_run batchs
        # trainer = L.Trainer(limit_train_batches=50, max_epochs=1, default_root_dir='checkpoints_kfold/', logger=logger) # usar apenas essa % dos dados de cada lote de treino
    
        trainer = L.Trainer(accelerator='gpu', max_epochs=num_epochs, logger=logger, callbacks=[timer])
        
        # treino
        lmodel = ClassificationModule(model, loss_function, torch.optim.Adam, fold)
        trainer.fit(model=lmodel, train_dataloaders=trainloader)
        print(timer.time_elapsed("train"))
        
        # teste 
        trainer.test(model=lmodel, dataloaders=testloader) # test na fold de test
        print(timer.end_time("test"))
        trainer.test(model=lmodel, dataloaders=idridloader) # test no idrid
        print(timer.end_time("test")) 

In [3]:
# Leitura IDRID
train_idrid_dataset = IDRIDDataset(IDRID_DIR/'1. Original Images/a. Training Set',
                                   IDRID_DIR/'2. Groundtruths/a. IDRiD_Disease Grading_Training Labels.csv',
                                   data_transforms['test'],
                                   convert_to_binary=True,
                                   dataset_name='idrid')
test_idrid_dataset = IDRIDDataset(IDRID_DIR/'1. Original Images/b. Testing Set',
                                  IDRID_DIR/'2. Groundtruths/b. IDRiD_Disease Grading_Testing Labels.csv',                   
                                  data_transforms['test'],
                                  convert_to_binary=True,
                                  dataset_name='idrid')
idrid_dataset = ConcatDataset([train_idrid_dataset, test_idrid_dataset])

### VGG16 instanciation

In [3]:
# Transfer learning (fine tuning):
# Instanciação VGG16_BN
weights = models.VGG16_BN_Weights.DEFAULT # melhores pesos até o momento
vgg16 = models.vgg16_bn(weights=weights)
# transformações específicas da rede
vgg16_preprocess = weights.transforms()
# modificando a camada de classificação
num_ftrs = vgg16.classifier[-1].in_features
vgg16.classifier[-1] = nn.Linear(num_ftrs, num_classes)

In [4]:
vgg16_preprocess

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [5]:
vgg16.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=2, bias=True)
)

In [6]:
binary_train_and_test(vgg16, 'vgg16_bn')

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ./checkpoints_kfold/tb_logs/FOLD 0
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


FOLD 0
--------------------------------



  | Name          | Type             | Params
---------------------------------------------------
0 | model         | VGG              | 134 M 
1 | loss_module   | CrossEntropyLoss | 0     
2 | metrics_train | MetricCollection | 0     
3 | metrics_test  | MetricCollection | 0     
---------------------------------------------------
134 M     Trainable params
0         Non-trainable params
134 M     Total params
537.109   Total estimated model params size (MB)


Training: |                                                                                                   …

`Trainer.fit` stopped: `max_epochs=25` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Testing: |                                                                                                    …

[[1170   95]
 [  45 1184]]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      ddr-test-acc          0.9438652992248535
     ddr-test-aucroc        0.9888440370559692
       ddr-test-f1          0.9441786408424377
      ddr-test-loss         0.15066899359226227
   ddr-test-precision       0.9257231950759888
     ddr-test-recall        0.9633848667144775
  ddr-test-specificity      0.9249011874198914
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |                                                                                                    …

[[ 36 132]
 [  2 346]]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     idrid-test-acc         0.7403100728988647
    idrid-test-aucroc       0.9635758399963379
      idrid-test-f1         0.8377723693847656
     idrid-test-loss        1.6092748641967773
  idrid-test-precision       0.723849356174469
    idrid-test-recall       0.9942528605461121
 idrid-test-specificity     0.2142857164144516
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
2703.3201091910014
251236.223690297


### ResNet50 Instanciation

In [6]:
res50_best_weights = models.ResNet50_Weights.DEFAULT
res50_model = models.resnet50(weights=res50_best_weights)
num_ftrs = res50_model.fc.in_features
res50_model.fc = nn.Linear(num_ftrs, num_classes)
res50_preprocess = res50_best_weights.transforms()

In [7]:
res50_preprocess

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [8]:
res50_model.fc 

Linear(in_features=2048, out_features=2, bias=True)

In [23]:
binary_train_and_test(res50_model, 'ResNet50')

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ./checkpoints_kfold/tb_logs/FOLD 0 ResNet50
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | model         | ResNet           | 23.5 M
1 | loss_module   | CrossEntropyLoss | 0     
2 | metrics_train | MetricCollection | 0     
3 | metrics_test  | MetricCollection | 0     
---------------------------------------------------
23.5 M    Trainable params
0         Non-trainable params
23.5 M    Total params
94.049    Total estimated model params size (MB)


FOLD 0
--------------------------------


Training: |                                                                                                   …

`Trainer.fit` stopped: `max_epochs=25` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


2681.4140807199874


...

[[1227   38]
 [  69 1160]]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      ddr-test-acc           0.957097053527832
     ddr-test-aucroc        0.9918677806854248
       ddr-test-f1          0.9559126496315002
      ddr-test-loss         0.12521512806415558
   ddr-test-precision       0.9682804942131042
     ddr-test-recall        0.9438567757606506
  ddr-test-specificity      0.9699604511260986
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


270314.651294326


Testing: |                                                                                                    …

[[ 25 143]
 [  0 348]]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     idrid-test-acc         0.7228682041168213
    idrid-test-aucroc       0.9378848075866699
      idrid-test-f1         0.8295590281486511
     idrid-test-loss         2.096717119216919
  idrid-test-precision      0.7087576389312744
    idrid-test-recall               1.0
 idrid-test-specificity     0.1488095223903656
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
270328.542583761


### MobileNet v2 Instanciation

In [5]:
mobi_best_weights = models.MobileNet_V2_Weights.DEFAULT
mobi_model = models.mobilenet_v2(weights=mobi_best_weights)
mobi_preprocess = mobi_best_weights.transforms()
num_ftrs = mobi_model.classifier[-1].in_features
mobi_model.classifier[-1] = nn.Linear(num_ftrs, num_classes)

In [7]:
binary_train_and_test(mobi_model, 'MobileNetV2')

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ./checkpoints_kfold/tb_logs/FOLD 0 MobileNetV2
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | model         | MobileNetV2      | 2.2 M 
1 | loss_module   | CrossEntropyLoss | 0     
2 | metrics_train | MetricCollection | 0     
3 | metrics_test  | MetricCollection | 0     
---------------------------------------------------
2.2 M     Trainable params
0         Non-trainable params
2.2 M     Total params
8.906     Total estimated model params size (MB)


FOLD 0
--------------------------------


Training: |                                                                                                   …

`Trainer.fit` stopped: `max_epochs=25` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


2672.32289527799


...

[[1227   38]
 [  66 1163]]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      ddr-test-acc          0.9582999348640442
     ddr-test-aucroc        0.9942869544029236
       ddr-test-f1          0.9572016596794128
      ddr-test-loss         0.10830139368772507
   ddr-test-precision       0.9683597087860107
     ddr-test-recall         0.946297824382782
  ddr-test-specificity      0.9699604511260986
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


284109.854408468


Testing: |                                                                                                    …

[[ 48 120]
 [  6 342]]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     idrid-test-acc         0.7558139562606812
    idrid-test-aucroc       0.9222171306610107
      idrid-test-f1         0.8444444537162781
     idrid-test-loss        1.6882370710372925
  idrid-test-precision      0.7402597665786743
    idrid-test-recall        0.982758641242981
 idrid-test-specificity     0.2857142984867096
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
284123.81001955


### EfficientNetB7 INstanciation

In [4]:
effcb7_best_weights = models.EfficientNet_B7_Weights.DEFAULT
effcb7_model = models.efficientnet_b7(weights=effcb7_best_weights)
effcb7_preprocess = effcb7_best_weights.transforms()
num_ftrs = effcb7_model.classifier[-1].in_features
effcb7_model.classifier[-1] = nn.Linear(num_ftrs, num_classes)

In [5]:
binary_train_and_test(effcb7_model, 'EfficientNetB7')

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | model         | EfficientNet     | 63.8 M
1 | loss_module   | CrossEntropyLoss | 0     
2 | metrics_train | MetricCollection | 0     
3 | metrics_test  | MetricCollection | 0     
---------------------------------------------------
63.8 M    Trainable params
0         Non-trainable params
63.8 M    Total params
255.168   Total estimated model params size (MB)


FOLD 0
--------------------------------


Training: |                                                                                                   …

OutOfMemoryError: CUDA out of memory. Tried to allocate 24.00 MiB. GPU 0 has a total capacity of 11.72 GiB of which 37.50 MiB is free. Including non-PyTorch memory, this process has 11.33 GiB memory in use. Of the allocated memory 10.74 GiB is allocated by PyTorch, and 368.88 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

### EfficientNetV2-L INstanciation

In [4]:
effcv2l_best_weights = models.EfficientNet_V2_L_Weights.DEFAULT
effcv2l_model = models.efficientnet_v2_l(weights=effcv2l_best_weights)
effcv2l_preprocess = effcv2l_best_weights.transforms()
num_ftrs = effcv2l_model.classifier[-1].in_features
effcv2l_model.classifier[-1] = nn.Linear(num_ftrs, num_classes)

In [5]:
effcv2l_preprocess

ImageClassification(
    crop_size=[480]
    resize_size=[480]
    mean=[0.5, 0.5, 0.5]
    std=[0.5, 0.5, 0.5]
    interpolation=InterpolationMode.BICUBIC
)

In [6]:
effcv2l_model.classifier

Sequential(
  (0): Dropout(p=0.4, inplace=True)
  (1): Linear(in_features=1280, out_features=2, bias=True)
)

In [7]:
binary_train_and_test(effcv2l_model, 'EfficientNetV2-L')

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


FOLD 0
--------------------------------


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 11.72 GiB of which 17.19 MiB is free. Process 689449 has 11.11 GiB memory in use. Including non-PyTorch memory, this process has 290.00 MiB memory in use. Of the allocated memory 93.23 MiB is allocated by PyTorch, and 4.77 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)